In [6]:
import cv2
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import os

def gray_scale(img):
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_thresh, high_thresh):
    return cv2.Canny(img, low_thresh, high_thresh)
    
def gaussianBlur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_Of_Intrest(img, vertices):
    
    #defining a blank mask to start with
    mask= np.zeros_like(img)
    
    #defining 3 or 1 channel color to fill the mask with depending on the input image
    if(len(img.shape)> 2):
        
        channel_count= img.shape[2] # i.e 3 or 4 depending upon the image
        ignore_mask_color= (255,)*channel_count
        
    else:
        ignore_mask_color= 255
        
        
    #filling pixels inside the polygon defined by the "vertices" with the color
    cv2.fillPoly(img, vertices, ignore_mask_color)
    
    #returning the image only where pixels are non-zero
    masked_img= cv2.bitwise_and(img, mask)
    
    return masked_img

def draw_Lines(img, lines, color=[255,0,0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2,y2), color, thickness)
            
    return img

def hough_Line(img, rho, theta, threshold, min_line_length, max_line_gap):
    lines= cv2.HoughLines(img, rho, theta, threshold, min_line_length, max_line_gap)
    line_img= np.zeros((img.shape[0], img.shape[1],3), dtype= np.uint8)
    draw_line(line_img,lines)
    
    return line_img

def weighnted_img(img,initial_img, alpha= 0.8, beta=1.,gama=0.):
    return cv2.addWeighted(initial_img, alpha, img, beta, gama)

def read_img(img):
    return mpimg.imread(img)

def to_HLS(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

def to_HSV(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

def isolated_color_mask(img, low_thresh, upper_thresh):
    assert(low_thresh.all()>=0 and low_thresh.all()<=255)
    assert(upper_thresh.all()>=0 and upper_thresh.all()<=255)
    
    return cv2.inRange(img, low_thresh, upper_thresh)

def adjust_Gamma(img, gamma= 1.0):
    invGamma= 1.0/gamma
    table= np.array([((i/255.0)** invGamma)*255 for i in np.arange(0, 256)]).astype("uint8")
    
    #apply gamma correction using Gamma lookup table
    return cv2.LUT(image, table)

def save_Image(img_list, labels, prefix= 'Test', op_folder= 'test_img_output'):
    if not os.path.exists(op_folder):
        os.mkdir(op_folder)
        
    for img, label in zip(img_list, labels):
        PATH= op_folder + "/"+prefix+'_'+label
        Image.fromarray(img).save(PATH)
        

def display_Imgs(img_list, labels=[], cols=2, fig_size=(15,15)):
    if len(labels)>0:
        assert(len(img_list)==len(labels))
        
    assert(len(img_list)>0)
    cmap= None
    tot= len(img_list)
    rows= int(tot/cols)
    plt.figure(figsize=fig_size)
    
    for i in range(tot):
        plt.subplot(rows, cols, i+1)
        
        if len(img_list[i].shape)==2:
            cmap='gray'
            
        if len(labels)>0:
            plt.title(labels[i])
          
        plt.imshow(img_list[i], cmap=cmap)
        
    plt.tight_layout()
    plt.show()

        
def get_Aoi(img):
    rows, cols= img.shape[:2]
    mask= np.zeros_like(img)
    
    left_bottom= [cols*0.1, rows]
    right_bottom= [cols*0.95, rows]
    left_top= [cols*0.4, rows*0.6]
    right_top= [cols*0.6, rows*0.6]
    
    vertices= np.array([[left_bottom, left_top, right_top, right_bottom]], dtype= np.int32)
    
    if len(mask.shape)==2:
        cv2.fillPoly(mask, vertices, 255)
        
    else:
        cv2.fillPoly(mask, vertices, (255,)*mask.shape[2])
        
    return cv2.bitwise_and(img, mask)

def get_Houg_Lines(img, rho=1, theta= np.pi/180, threshold= 20, min_len=20, max_gap= 300):
    lines= cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_len,  maxLineGap=max_gap)
    
    return lines

def get_Line_Length(line):
    for x1, y1, x2, y2 in line:
        return np.sqrt((y2-y1)**2 + (x2-x1)**2)
    
def get_Line_Slope_Intercept(line):
    for x1, y1, x2, y2 in line:
        if x2-x1==0:
            return math.inf, 0
        
        slope= (y2-y1)/(x2-x1)
        intercept= y1 - slope*x1
        
    return slope, intercept

def get_Line_Slope_Intercept(lines):
    left_lines = []
    right_lines = []
    left_lengths = []
    right_lengths = []
    
    for line in lines:
        slope, intercept= get_Line_Slope_Intercept(line)
        if slope== math.inf:
            continue
        line_len= get_Line_Length(line)
        
        if slope<0:
            left_lines.append((slope, intercept))
            left_lengths.append(line_len)
            
        else:
            right_lines.append((slope, intercept))
            right_lengths.append(line_len)
            
        # average
        left_avg= np.dot(left_lines,left_lengths)/np.sum(left_lengths) if len(left_lengths)>0 else null
        right_avg= np.dot(right_lines, right_lengths)/np.sum(right_lengths) if len(right_lengths)>0 else null
        
        return left_avg, right_avg
    
def convert_slope_intercept_to_line(y1, y2 , line):
    if line is null:
        return null
    
    slope, intercept = line
    x1 = int((y1- intercept)/slope)
    y1 = int(y1)
    x2 = int((y2- intercept)/slope)
    y2 = int(y2)
    return((x1, y1),(x2, y2))


def get_lane_lines(img, lines):
    left_avg, right_avg = get_Line_Slope_Intercept(lines)
    
    y1 = img.shape[0]
    y2 = img.shape[0] * 0.6
    
    left_lane = convert_slope_intercept_to_line(y1, y2, left_avg)
    right_lane = convert_slope_intercept_to_line(y1, y2, right_avg)
    return left_lane, right_lane

def draw_weighted_lines(img, lines, color=[255, 0, 0], thickness=2, alpha = 1.0, beta = 0.95, gamma= 0):
    mask_img = np.zeros_like(img)
    for line in lines:
        if line is not null:
            cv2.line(mask_img, *line, color, thickness)            
    return weighted_img(mask_img, img, alpha, beta, gamma)